In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Turkish Product Reviews

### Dataset Preprocessing

In [ ]:
import os
import sys
from utils import missing_values
import pandas as pd
import numpy as np
from datasets import load_dataset
import time
from datetime import datetime
import warnings

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

#### Set Path and Constant Values

In [ ]:
# Set Constants
ROOT_PATH = "/content/drive/MyDrive/Colab Notebooks/HW2"  # Google Drive için kök dizin
CFG_PATH = os.path.join(ROOT_PATH, 'cfg')
DATA_PATH = os.path.join(ROOT_PATH, 'data')
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw')
EMBEDDING_DATA_PATH = os.path.join(DATA_PATH, 'embedding')
sys.path.append(os.path.join(ROOT_PATH, 'src'))

# Create Folders
os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(EMBEDDING_DATA_PATH, exist_ok=True)

### Import Dataset: HepsiBurada Product Reviews

In [ ]:
df_hb_data_all = pd.read_parquet("hf://datasets/fthbrmnby/turkish_product_reviews/data/train-00000-of-00001.parquet")
df_hb_data_all.to_csv(os.path.join(RAW_DATA_PATH, "hb_tr_product_reviews.csv.gz"), index=False, compression='gzip')
# dataset = load_dataset("fthbrmnby/turkish_product_reviews", split="train")
# df_hb_data_all = dataset.to_pandas()

df_hb_data_all.head()

,sentence,sentiment
0,fena değil paraya göre iyi.,1
1,ürün kaliteli ve çok kullanışlı kargo ya verilmesi ve elime çabuk ulaşması da gayet başarılı idi,1
2,fiyat-performans karşılaştırması yaptığınızda gayet iyi bir ürün. en önemlisi ısınma yapmıyor.,1
3,bu fiyata çok iyi,1
4,fiyat ve performans olarak gayet iyi,1


### Show Metadata, Shape, Statistics and Missing Table

In [ ]:
df_hb_data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235165 entries, 0 to 235164
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sentence   235165 non-null  object
 1   sentiment  235165 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [ ]:
df_hb_data_all.shape

(235165, 2)

In [ ]:
df_hb_data_all['sentiment'].value_counts(normalize=True)

,proportion
sentiment,
1,0.936721
0,0.063279


In [ ]:
missing_values(df=df_hb_data_all, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
sentence,0,0.0
sentiment,0,0.0


In [ ]:
df_hb_data_all.head(5)

,sentence,sentiment
0,fena değil paraya göre iyi.,1
1,ürün kaliteli ve çok kullanışlı kargo ya verilmesi ve elime çabuk ulaşması da gayet başarılı idi,1
2,fiyat-performans karşılaştırması yaptığınızda gayet iyi bir ürün. en önemlisi ısınma yapmıyor.,1
3,bu fiyata çok iyi,1
4,fiyat ve performans olarak gayet iyi,1


### Split Data

In [ ]:
# Neden Yapıyoruz
# 1- Balanced Dataset
# 2- Computing Power

In [ ]:
# Pozitif (1) ve negatif (0) sınıfları dengeli bir şekilde seçiyoruz.
# Her sınıftan 3750 örnek alıyoruz.
df_hb_data_1 = df_hb_data_all[df_hb_data_all['sentiment'] == 1].sample(n=3750, random_state=34)
df_hb_data_0 = df_hb_data_all[df_hb_data_all['sentiment'] == 0].sample(n=3750, random_state=34)

# Pozitif ve negatif sınıfları birleştirerek dengeli bir veri seti oluşturuyoruz.
df_hb_data_balanced = pd.concat([df_hb_data_1, df_hb_data_0])

# Veri setini karıştırıyoruz (shuffle) ve index'leri sıfırlıyoruz.
df_hb_data = df_hb_data_balanced.sample(frac=1, random_state=34).reset_index(drop=True)

# Sınıf dağılımını kontrol ediyoruz.
print(df_hb_data['sentiment'].value_counts())

sentiment
1    3750
0    3750
Name: count, dtype: int64


In [ ]:
df_hb_data["sentenceLength"] = df_hb_data["sentence"].apply(len)
df_hb_data["sentenceWordCount"] = df_hb_data['sentence'].str.split().str.len()

In [ ]:
df_hb_data.head(10)

,sentence,sentiment,sentenceLength,sentenceWordCount
0,mükemmel bir filtre kağıdı....teşekkürler....,1,45,4
1,"ürün bugün elime geçti, yorgan battaniye hepsini poşetledim çok az yer kaplıyor ve yorganlar tertemiz kalıyor en önemlisi de bu",1,127,20
2,cihaz çok güzel ve şık satıcı nasıl yaptıysa taa adananın rize ye 24 saat dolmadan cihazı ulaştırdı helal olsun çok teşekkürler,1,127,21
3,fakat ben bu saati̇ ona söylemeden aldim. süpri̇z yapmak i̇stedi̇m. ve i̇yi̇ki̇de bu saati̇ almişim.oğlum çok beğendi̇. güle güle kullansin. almak i̇steyenlerde bu firati kaçirmasinlar. hepsi̇buradaya çok teşekkürler.,1,217,28
4,"yeni oluşturulacak telefon kayıtları telefon hafızası ve sim karta kayıt yapmıyor, gmail adresine gönderiyor",0,108,14
5,üstüne tuvalet kağıdı tanımam çok iyi çok süper varya,1,53,9
6,"bu ürünle bir fön makinesi arasında kalmıştım ve yorumlardan yola çıkarak bu ürünü almıştım ama pek bana göre olmadığını gördüm,yaklaşık 5 aydır kullanıyorum .saçlarım az olmasına rağmen saçlarımın kuruması yarım saati buluyor en yüksek seviyede kullansam bile.satin özelliğini açarsam süre daha da uzuyor.eskiden kullandığım makinelere göre performansı çok düşük.tek olumlu yanı saçlar kuruduktan sonra tek tek havalanmıyor.",0,425,57
7,bisiklet kullanicilari icin guzel bir urun,1,42,6
8,sürekli̇ kkullandiğim traş jeli̇ memnunum,1,41,5
9,"kardeşime hediye aldım bu ürünü, fakat sadece bacaklar için çok ideal bir ürün onun dışında ayrı aparatları olmadığı için daha küçük bölgeler için uygun bir ürün değil, işlevine göre fiyatı da makul, işlevli ürün aramayanlar için gayet ideal bir ürün...",1,253,40


In [ ]:
df_hb_data[["sentenceWordCount"]].describe(percentiles=[0.80,0.85,0.90,0.95,0.99]).T

,count,mean,std,min,50%,80%,85%,90%,95%,99%,max
sentenceWordCount,7500.0,22.0956,18.370803,0.0,19.0,32.0,36.0,42.0,55.0,93.01,276.0


In [ ]:
df_hb_data.groupby(by=['sentiment'], as_index=False).agg(SentimentWordCount_Mean=('sentenceWordCount','mean'),
                                                         SentimentLength_Mean=('sentenceLength','mean'))

,sentiment,SentimentWordCount_Mean,SentimentLength_Mean
0,0,22.204267,156.734667
1,1,21.986933,156.841067


### Embedding

In [ ]:
# Sadece gerekli sütunları kullanarak veri setini hazırlıyoruz
df = df_hb_data[['sentence', 'sentiment']]
# İlk 5 satırı kontrol ediyoruz (örnek embedding için)
df[0:5]

,sentence,sentiment
0,mükemmel bir filtre kağıdı....teşekkürler....,1
1,"ürün bugün elime geçti, yorgan battaniye hepsini poşetledim çok az yer kaplıyor ve yorganlar tertemiz kalıyor en önemlisi de bu",1
2,cihaz çok güzel ve şık satıcı nasıl yaptıysa taa adananın rize ye 24 saat dolmadan cihazı ulaştırdı helal olsun çok teşekkürler,1
3,fakat ben bu saati̇ ona söylemeden aldim. süpri̇z yapmak i̇stedi̇m. ve i̇yi̇ki̇de bu saati̇ almişim.oğlum çok beğendi̇. güle güle kullansin. almak i̇steyenlerde bu firati kaçirmasinlar. hepsi̇buradaya çok teşekkürler.,1
4,"yeni oluşturulacak telefon kayıtları telefon hafızası ve sim karta kayıt yapmıyor, gmail adresine gönderiyor",0


In [ ]:
from sentence_transformers import SentenceTransformer

# Embedding modelleri listesi
embedding_model = [
    'sentence-transformers/all-MiniLM-L12-v2',
    "jinaai/jina-embeddings-v3",
    "intfloat/multilingual-e5-large-instruct",
    "BAAI/bge-m3",
    "thenlper/gte-large"
]

In [ ]:
for model_name in embedding_model:
    try:
        embedding_duration = datetime.now()
        print(f"Embedding Model Name: {model_name} - {datetime.now()} ")
        model = SentenceTransformer(model_name, trust_remote_code=True)
        model_output_file = os.path.join(
            EMBEDDING_DATA_PATH, f"hb_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        embeddings = model.encode(df['sentence'].tolist())
        embedding_dim = len(embeddings[0])
        print(f"Embedding Dimension: {embedding_dim}")
        embedding_columns = [f'embedding_{i}' for i in range(embedding_dim)]
        df_embedding = pd.DataFrame(embeddings, columns=embedding_columns)
        df_data = pd.concat([df, df_embedding], axis=1)
        df_data.to_csv(model_output_file, compression='gzip', index=False)
        print(f"Embedding Duration: {datetime.now() - embedding_duration}")
        print("- " * 15)
    except Exception as exc:
        print("Exception: ", exc)

Embedding Model Name: sentence-transformers/all-MiniLM-L12-v2 - 2024-11-28 18:06:42.030438 
Embedding Dimension: 384
Embedding Duration: 0:00:21.367799
- - - - - - - - - - - - - - - 
Embedding Model Name: jinaai/jina-embeddings-v3 - 2024-11-28 18:07:03.398379 


Embedding Dimension: 1024
Embedding Duration: 0:00:50.279299
- - - - - - - - - - - - - - - 
Embedding Model Name: intfloat/multilingual-e5-large-instruct - 2024-11-28 18:07:53.677882 
Embedding Dimension: 1024
Embedding Duration: 0:01:17.070758
- - - - - - - - - - - - - - - 
Embedding Model Name: BAAI/bge-m3 - 2024-11-28 18:09:10.748816 
Embedding Dimension: 1024
Embedding Duration: 0:01:17.616235
- - - - - - - - - - - - - - - 
Embedding Model Name: thenlper/gte-large - 2024-11-28 18:10:28.365182 
Embedding Dimension: 1024
Embedding Duration: 0:01:32.163953
- - - - - - - - - - - - - - - 


In [ ]:
df_emb_data_1 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "hb_sentence-transformers__all-MiniLM-L12-v2_embeddings.csv.gz"), compression='gzip')
df_emb_data_1

,sentence,sentiment,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,embedding_10,embedding_11,embedding_12,embedding_13,embedding_14,embedding_15,embedding_16,embedding_17,embedding_18,embedding_19,embedding_20,embedding_21,embedding_22,embedding_23,embedding_24,embedding_25,embedding_26,embedding_27,embedding_28,embedding_29,embedding_30,embedding_31,embedding_32,embedding_33,embedding_34,embedding_35,embedding_36,embedding_37,embedding_38,embedding_39,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49,embedding_50,embedding_51,embedding_52,embedding_53,embedding_54,embedding_55,embedding_56,embedding_57,embedding_58,embedding_59,embedding_60,embedding_61,embedding_62,embedding_63,embedding_64,embedding_65,embedding_66,embedding_67,embedding_68,embedding_69,embedding_70,embedding_71,embedding_72,embedding_73,embedding_74,embedding_75,embedding_76,embedding_77,embedding_78,embedding_79,embedding_80,embedding_81,embedding_82,embedding_83,embedding_84,embedding_85,embedding_86,embedding_87,embedding_88,embedding_89,embedding_90,embedding_91,embedding_92,embedding_93,embedding_94,embedding_95,embedding_96,embedding_97,embedding_98,embedding_99,embedding_100,embedding_101,embedding_102,embedding_103,embedding_104,embedding_105,embedding_106,embedding_107,embedding_108,embedding_109,embedding_110,embedding_111,embedding_112,embedding_113,embedding_114,embedding_115,embedding_116,embedding_117,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127,embedding_128,embedding_129,embedding_130,embedding_131,embedding_132,embedding_133,embedding_134,embedding_135,embedding_136,embedding_137,embedding_138,embedding_139,embedding_140,embedding_141,embedding_142,embedding_143,embedding_144,embedding_145,embedding_146,embedding_147,embedding_148,embedding_149,embedding_150,embedding_151,embedding_152,embedding_153,embedding_154,embedding_155,embedding_156,embedding_157,embedding_158,embedding_159,embedding_160,embedding_161,embedding_162,embedding_163,embedding_164,embedding_165,embedding_166,embedding_167,embedding_168,embedding_169,embedding_170,embedding_171,embedding_172,embedding_173,embedding_174,embedding_175,embedding_176,embedding_177,embedding_178,embedding_179,embedding_180,embedding_181,embedding_182,embedding_183,embedding_184,embedding_185,embedding_186,embedding_187,embedding_188,embedding_189,embedding_190,embedding_191,embedding_192,embedding_193,embedding_194,embedding_195,embedding_196,embedding_197,embedding_198,embedding_199,embedding_200,embedding_201,embedding_202,embedding_203,embedding_204,embedding_205,embedding_206,embedding_207,embedding_208,embedding_209,embedding_210,embedding_211,embedding_212,embedding_213,embedding_214,embedding_215,embedding_216,embedding_217,embedding_218,embedding_219,embedding_220,embedding_221,embedding_222,embedding_223,embedding_224,embedding_225,embedding_226,embedding_227,embedding_228,embedding_229,embedding_230,embedding_231,embedding_232,embedding_233,embedding_234,embedding_235,embedding_236,embedding_237,embedding_238,embedding_239,embedding_240,embedding_241,embedding_242,embedding_243,embedding_244,embedding_245,embedding_246,embedding_247,embedding_248,embedding_249,embedding_250,embedding_251,embedding_252,embedding_253,embedding_254,embedding_255,embedding_256,embedding_257,embedding_258,embedding_259,embedding_260,embedding_261,embedding_262,embedding_263,embedding_264,embedding_265,embedding_266,embedding_267,embedding_268,embedding_269,embedding_270,embedding_271,embedding_272,embedding_273,embedding_274,embedding_275,embedding_276,embedding_277,embedding_278,embedding_279,embedding_280,embedding_281,embedding_282,embedding_283,embedding_284,embedding_285,embedding_286,embedding_287,embedding_288,embedding_289,embedding_290,embedding_291,em

In [ ]:
df_emb_data_2 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "hb_jinaai__jina-embeddings-v3_embeddings.csv.gz"), compression='gzip')
df_emb_data_2

sentence  \
0                                                                                                                                                                                                                                                                                                                                                               mükemmel bir filtre kağıdı....teşekkürler....   
1                                                                                                                                                                                                                                                                             ürün bugün elime geçti, yorgan battaniye hepsini poşetledim çok az yer kaplıyor ve yorganlar tertemiz kalıyor en önemlisi de bu   
2                                                                                                                                                                                                                                                                             cihaz çok güzel ve şık satıcı nasıl yaptıysa taa adananın rize ye 24 saat dolmadan cihazı ulaştırdı helal olsun çok teşekkürler   
3                                                                                                                                                                                   fakat ben bu saati̇ ona söylemeden aldim. süpri̇z yapmak i̇stedi̇m. ve i̇yi̇ki̇de bu saati̇ almişim.oğlum çok beğendi̇. güle güle kullansin. almak i̇steyenlerde bu firati kaçirmasinlar. hepsi̇buradaya çok teşekkürler.   
4                                                                                                                                                                                                                                                                                                yeni oluşturulacak telefon kayıtları telefon hafızası ve sim karta kayıt yapmıyor, gmail adresine gönderiyor   
...                                                                                                                                                                                                                                                                                                                                                                                                       ...   
7495  buradaki yorumları alıp okuduktan sonra almıştım bu ürünü bu yüzden suyu birden ve tazyikli açmıyorum, yavaş yavaş ve itina ile de kullanınca sorun çıkarmıyor.. ancak ekstra conta kullanmama rağmen suyu normal seviyelerde kullanınca yine de sulama tabancasının altından su damlatıyor.. bu fiyata yine de makul bir ürün.. çok fazla beklentiniz yoksa hor kullanmayacaksanız iş görür bir ürün..   
7496                                                                                                                                                                                                                              o kadar kaliteli değil beklediğimden daha kötü 1 tusu basılı kalıyor mecburen o tuşun altındaki plastiği kullanmadıgınız bir tuş ile değiştirmeniz gerekiyor mouse ise kötü   
7497                                                                                                                                                                                                çay seti almaya karar verdiğimizde en çok yorumları okuyarak arçelik in bu modelini beğendik. i̇yiki bu modele karar vermiştik herşeyi  mükemmel hepsiburdayada zamanında teslim ettiği için teşekkür ler   
7498                                      hepsiburada yine sasirtmadi. guvenli kutu hizli kargo. ancak ürün ile isinamadik. konusma sirasinda bir problem yokken muzik dinlerken ses surekli kesik kesik geliyor. telefonla uyumsuzluk soz konusu olabilecegi yazilmis yabanci sitelerde, note 2 kullanan arkadaslar eger bir cozum bulannvarsa ismimle herhangi bir sosyal agdan ulasmalar

In [ ]:
df_emb_data_3 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "hb_intfloat__multilingual-e5-large-instruct_embeddings.csv.gz"), compression='gzip')
df_emb_data_3

sentence  \
0                                                                                                                                                                                                                                                                                                                                                               mükemmel bir filtre kağıdı....teşekkürler....   
1                                                                                                                                                                                                                                                                             ürün bugün elime geçti, yorgan battaniye hepsini poşetledim çok az yer kaplıyor ve yorganlar tertemiz kalıyor en önemlisi de bu   
2                                                                                                                                                                                                                                                                             cihaz çok güzel ve şık satıcı nasıl yaptıysa taa adananın rize ye 24 saat dolmadan cihazı ulaştırdı helal olsun çok teşekkürler   
3                                                                                                                                                                                   fakat ben bu saati̇ ona söylemeden aldim. süpri̇z yapmak i̇stedi̇m. ve i̇yi̇ki̇de bu saati̇ almişim.oğlum çok beğendi̇. güle güle kullansin. almak i̇steyenlerde bu firati kaçirmasinlar. hepsi̇buradaya çok teşekkürler.   
4                                                                                                                                                                                                                                                                                                yeni oluşturulacak telefon kayıtları telefon hafızası ve sim karta kayıt yapmıyor, gmail adresine gönderiyor   
...                                                                                                                                                                                                                                                                                                                                                                                                       ...   
7495  buradaki yorumları alıp okuduktan sonra almıştım bu ürünü bu yüzden suyu birden ve tazyikli açmıyorum, yavaş yavaş ve itina ile de kullanınca sorun çıkarmıyor.. ancak ekstra conta kullanmama rağmen suyu normal seviyelerde kullanınca yine de sulama tabancasının altından su damlatıyor.. bu fiyata yine de makul bir ürün.. çok fazla beklentiniz yoksa hor kullanmayacaksanız iş görür bir ürün..   
7496                                                                                                                                                                                                                              o kadar kaliteli değil beklediğimden daha kötü 1 tusu basılı kalıyor mecburen o tuşun altındaki plastiği kullanmadıgınız bir tuş ile değiştirmeniz gerekiyor mouse ise kötü   
7497                                                                                                                                                                                                çay seti almaya karar verdiğimizde en çok yorumları okuyarak arçelik in bu modelini beğendik. i̇yiki bu modele karar vermiştik herşeyi  mükemmel hepsiburdayada zamanında teslim ettiği için teşekkür ler   
7498                                      hepsiburada yine sasirtmadi. guvenli kutu hizli kargo. ancak ürün ile isinamadik. konusma sirasinda bir problem yokken muzik dinlerken ses surekli kesik kesik geliyor. telefonla uyumsuzluk soz konusu olabilecegi yazilmis yabanci sitelerde, note 2 kullanan arkadaslar eger bir cozum bulannvarsa ismimle herhangi bir sosyal agdan ulasmalar

In [ ]:
df_emb_data_4 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "hb_BAAI__bge-m3_embeddings.csv.gz"), compression='gzip')
df_emb_data_4

sentence  \
0                                                                                                                                                                                                                                                                                                                                                               mükemmel bir filtre kağıdı....teşekkürler....   
1                                                                                                                                                                                                                                                                             ürün bugün elime geçti, yorgan battaniye hepsini poşetledim çok az yer kaplıyor ve yorganlar tertemiz kalıyor en önemlisi de bu   
2                                                                                                                                                                                                                                                                             cihaz çok güzel ve şık satıcı nasıl yaptıysa taa adananın rize ye 24 saat dolmadan cihazı ulaştırdı helal olsun çok teşekkürler   
3                                                                                                                                                                                   fakat ben bu saati̇ ona söylemeden aldim. süpri̇z yapmak i̇stedi̇m. ve i̇yi̇ki̇de bu saati̇ almişim.oğlum çok beğendi̇. güle güle kullansin. almak i̇steyenlerde bu firati kaçirmasinlar. hepsi̇buradaya çok teşekkürler.   
4                                                                                                                                                                                                                                                                                                yeni oluşturulacak telefon kayıtları telefon hafızası ve sim karta kayıt yapmıyor, gmail adresine gönderiyor   
...                                                                                                                                                                                                                                                                                                                                                                                                       ...   
7495  buradaki yorumları alıp okuduktan sonra almıştım bu ürünü bu yüzden suyu birden ve tazyikli açmıyorum, yavaş yavaş ve itina ile de kullanınca sorun çıkarmıyor.. ancak ekstra conta kullanmama rağmen suyu normal seviyelerde kullanınca yine de sulama tabancasının altından su damlatıyor.. bu fiyata yine de makul bir ürün.. çok fazla beklentiniz yoksa hor kullanmayacaksanız iş görür bir ürün..   
7496                                                                                                                                                                                                                              o kadar kaliteli değil beklediğimden daha kötü 1 tusu basılı kalıyor mecburen o tuşun altındaki plastiği kullanmadıgınız bir tuş ile değiştirmeniz gerekiyor mouse ise kötü   
7497                                                                                                                                                                                                çay seti almaya karar verdiğimizde en çok yorumları okuyarak arçelik in bu modelini beğendik. i̇yiki bu modele karar vermiştik herşeyi  mükemmel hepsiburdayada zamanında teslim ettiği için teşekkür ler   
7498                                      hepsiburada yine sasirtmadi. guvenli kutu hizli kargo. ancak ürün ile isinamadik. konusma sirasinda bir problem yokken muzik dinlerken ses surekli kesik kesik geliyor. telefonla uyumsuzluk soz konusu olabilecegi yazilmis yabanci sitelerde, note 2 kullanan arkadaslar eger bir cozum bulannvarsa ismimle herhangi bir sosyal agdan ulasmalar

In [ ]:
df_emb_data_5 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "hb_thenlper__gte-large_embeddings.csv.gz"), compression='gzip')
df_emb_data_5

sentence  \
0                                                                                                                                                                                                                                                                                                                                                               mükemmel bir filtre kağıdı....teşekkürler....   
1                                                                                                                                                                                                                                                                             ürün bugün elime geçti, yorgan battaniye hepsini poşetledim çok az yer kaplıyor ve yorganlar tertemiz kalıyor en önemlisi de bu   
2                                                                                                                                                                                                                                                                             cihaz çok güzel ve şık satıcı nasıl yaptıysa taa adananın rize ye 24 saat dolmadan cihazı ulaştırdı helal olsun çok teşekkürler   
3                                                                                                                                                                                   fakat ben bu saati̇ ona söylemeden aldim. süpri̇z yapmak i̇stedi̇m. ve i̇yi̇ki̇de bu saati̇ almişim.oğlum çok beğendi̇. güle güle kullansin. almak i̇steyenlerde bu firati kaçirmasinlar. hepsi̇buradaya çok teşekkürler.   
4                                                                                                                                                                                                                                                                                                yeni oluşturulacak telefon kayıtları telefon hafızası ve sim karta kayıt yapmıyor, gmail adresine gönderiyor   
...                                                                                                                                                                                                                                                                                                                                                                                                       ...   
7495  buradaki yorumları alıp okuduktan sonra almıştım bu ürünü bu yüzden suyu birden ve tazyikli açmıyorum, yavaş yavaş ve itina ile de kullanınca sorun çıkarmıyor.. ancak ekstra conta kullanmama rağmen suyu normal seviyelerde kullanınca yine de sulama tabancasının altından su damlatıyor.. bu fiyata yine de makul bir ürün.. çok fazla beklentiniz yoksa hor kullanmayacaksanız iş görür bir ürün..   
7496                                                                                                                                                                                                                              o kadar kaliteli değil beklediğimden daha kötü 1 tusu basılı kalıyor mecburen o tuşun altındaki plastiği kullanmadıgınız bir tuş ile değiştirmeniz gerekiyor mouse ise kötü   
7497                                                                                                                                                                                                çay seti almaya karar verdiğimizde en çok yorumları okuyarak arçelik in bu modelini beğendik. i̇yiki bu modele karar vermiştik herşeyi  mükemmel hepsiburdayada zamanında teslim ettiği için teşekkür ler   
7498                                      hepsiburada yine sasirtmadi. guvenli kutu hizli kargo. ancak ürün ile isinamadik. konusma sirasinda bir problem yokken muzik dinlerken ses surekli kesik kesik geliyor. telefonla uyumsuzluk soz konusu olabilecegi yazilmis yabanci sitelerde, note 2 kullanan arkadaslar eger bir cozum bulannvarsa ismimle herhangi bir sosyal agdan ulasmalar

In [ ]:
print(f"Model 1 Shape - MiniLM : {df_emb_data_1.shape}")
print(f"Model 2 Shape - Jina : {df_emb_data_2.shape}")
print(f"Model 3 Shape - E5-Large : {df_emb_data_3.shape}")
print(f"Model 4 Shape - BGE-M3 : {df_emb_data_4.shape}")
print(f"Model 5 Shape - GTE-Large : {df_emb_data_5.shape}")

Model 1 Embedding Shape: (7500, 386)
Model 2 Embedding Shape: (7500, 1026)
Model 3 Embedding Shape: (7500, 1026)
Model 4 Embedding Shape: (7500, 1026)
Model 5 Embedding Shape: (7500, 1026)
